In [10]:
%matplotlib notebook

In [11]:
import pyspark

sc = pyspark.sql.SparkSession.Builder().getOrCreate()

## Levanto los archivos

In [12]:
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from IPython.display import display, HTML
import math



## Utils

In [13]:
def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)


global x_0
global y_0

x_0 = 462385.503783397
y_0 = 6109042.35153865

def normalize_positions(row):
    row['x'] = row['x']-x_0
    row['y'] = row['y']-y_0
    return row



In [14]:
points_recep = sc.read.json('datos/train-test-by-emission.jsonlines/').rdd

In [15]:
def group_emisions(data):
    data_dict = data.asDict()
    recep_0 = data_dict.pop('recep_0')
    recep_1 = data_dict.pop('recep_1')
    recep_2 = data_dict.pop('recep_2')
    recep_3 = data_dict.pop('recep_3')
    
    data_dict['emissions'] = []
    for i in range(24):
        data_dict['emissions'].append([recep_0[i], recep_1[i], recep_2[i], recep_3[i]])
    return data_dict


In [16]:
points_emisions = points_recep.map(group_emisions)


In [17]:
def expand_rows_with_emissions(row):
    emissions = row.pop('emissions')
    print(row)
    rows = []
    for e in emissions:
        new_row = row.copy()
        new_row['recep'] = e
        rows.append(new_row)
    return rows
        
all_emissions = points_emisions.flatMap(expand_rows_with_emissions)

### Genero los atributos y etiquetas que me interesan

In [18]:
def generate_attrs(row):
    data = {
        'antenna_0': row['recep'][0],
        'antenna_1': row['recep'][1],
        'antenna_2': row['recep'][2],
        'antenna_3': row['recep'][3],
    }
    return {'data': data, 'x': row['x'], 'y': row['y'], 'point': row['Punto']}
    
all_emissions = all_emissions.map(generate_attrs)

### Saco las emisiones sin ninguna recepcion

In [19]:
all_emissions = all_emissions.filter(lambda x: sum(x['data'].values())>0)

### Normalizo coordenadas

In [20]:
all_emissions = all_emissions.map(lambda x: normalize_positions(x))

## Redes neuronales para regresion

In [21]:
def get_regressor_mae(predictions, real):
    mae = 0
    for i in range(len(predictions)):
        mae += abs(predictions[i]-real[i])
    mae = mae/len(predictions)
    return mae


In [28]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

In [29]:
regre_target = pd.DataFrame(all_emissions.map(lambda x: x['x']).collect())
regre_data = pd.DataFrame(all_emissions.map(lambda x: x['data']).collect())

regre_data_np = np.array(regre_data.astype(float))
regre_target_np = np.array(regre_target.astype(float))

In [30]:
class LinearRegressionModel(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__() 
        self.linear = nn.Linear(input_dim, output_dim, bias=True)
        
    def forward(self, x):
        # Here the forward pass is simply a linear function
        out = self.linear(x)
        return out



In [31]:
class BestModel(nn.Module):

    def __init__(self, input_dim, hidden_list, output_dim):

        super(BestModel, self).__init__() 
        # Calling Super Class's constructor
        self.linear1 = nn.Linear(input_dim, hidden_dim[0], bias=True)
        self.act = nn.Sigmoid()
        self.linear2 = nn.Linear(hidden_dim[-1], output_dim, bias=True)
        # nn.linear is defined in nn.Module

    def forward(self, x):
        # Here the forward pass is simply a linear function
        out = self.linear1(x)
        out = self.act(out)
        out = self.linear2(out)
        return out


In [32]:
MODEL_PATH = 'datos/tmp/nn-model'
def k_cross_validation(model_builder, data, target, k=5, init_from_file = False):   
    kf = GroupKFold(n_splits=k)
    mae_list = []
    mae_list_train = []
    k_index = 0
    
    unique_target = np.unique(target, axis=0)
    groups = []
    for h in range(len(target)):
        i,j = np.where(unique_target == target[h])
        groups.append(i[0])
    
    for train_index, test_index in kf.split(data, groups=groups):
        params = model_builder()
        model = params['model']
        if init_from_file:
            model.load_state_dict(torch.load(MODEL_PATH))
        l_rate = params['l_rate']
        optimiser = params['optimiser']
        criterion = params['criterion']
        epochs = params['epochs']

        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
        
        for epoch in range(epochs):

            epoch +=1
            #increase the number of epochs by 1 every time
            inputs = Variable(torch.from_numpy(X_train))
            labels = Variable(torch.from_numpy(y_train))

            inputs = torch.tensor(inputs, dtype=torch.float)
            labels = torch.tensor(labels, dtype=torch.float)

            #clear grads as discussed in prev post
            optimiser.zero_grad()
            #forward to get predicted values
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()# back props
            optimiser.step()# update the parameters
            if k_index == 0 and epoch < 1000: print('epoch {}, loss {}'.format(epoch,loss.item()))
            if k_index == 0 and epoch > 1000 and random.randint(0, 100) == 1: print('epoch {}, loss {}'.format(epoch,loss.item()))

        print(loss.data)
        
        test = Variable(torch.from_numpy(X_test))
        test = test.clone().detach().float()
        predicted = model.forward(test).data.numpy()
        mae = get_regressor_mae(predictions=predicted, real=y_test)
        print(mae)
        
        train = Variable(torch.from_numpy(X_train))
        train = train.clone().detach().float()
        predicted_train = model.forward(train).data.numpy()
        mae_train = get_regressor_mae(predictions=predicted_train, real=y_train)
        print(mae_train)
        
        mae_list.append(mae)
        mae_list_train.append(mae_train)
        k_index += 1
        #torch.save(model.state_dict(), MODEL_PATH)
        

    return mae_list, mae_list_train


In [36]:
def build_nn_params():
    #model = LinearRegressionModel(4,2)
    #model = BestModel(4, 4, 2)
    model = BestModel()
    criterion = nn.MSELoss()
    l_rate = 0.0000001
    optimiser = torch.optim.SGD(model.parameters(), lr = l_rate) 
    epochs = 300000
    
    return {
        'model': model,
        'l_rate': l_rate,
        'optimiser': optimiser,
        'criterion': criterion,
        'epochs': epochs
    }

In [37]:
class BestModel(nn.Module):

    def __init__(self):

        super(BestModel, self).__init__() 
        # Calling Super Class's constructor
        self.linear1 = nn.Linear(4, 8, bias=True)
        self.linear2 = nn.Linear(8, 1, bias=True)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.linear1(x))
        x = self.linear2(x)             
        return x


In [ ]:
k_cross_validation(build_nn_params, regre_data_np, regre_target_np, k=5, init_from_file=False)

<ipython-input-32-36ed0aad6efa>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float)
<ipython-input-32-36ed0aad6efa>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float)


epoch 1, loss 3904140.25
epoch 2, loss 3903682.5
epoch 3, loss 3903234.25
epoch 4, loss 3902797.0
epoch 5, loss 3902368.5
epoch 6, loss 3901949.0
epoch 7, loss 3901538.0
epoch 8, loss 3901134.75
epoch 9, loss 3900739.75
epoch 10, loss 3900351.0
epoch 11, loss 3899970.0
epoch 12, loss 3899595.75
epoch 13, loss 3899227.5
epoch 14, loss 3898865.0
epoch 15, loss 3898509.5
epoch 16, loss 3898158.5
epoch 17, loss 3897813.0
epoch 18, loss 3897472.0
epoch 19, loss 3897136.25
epoch 20, loss 3896803.5
epoch 21, loss 3896475.5
epoch 22, loss 3896150.5
epoch 23, loss 3895829.25
epoch 24, loss 3895510.75
epoch 25, loss 3895195.0
epoch 26, loss 3894881.75
epoch 27, loss 3894570.75
epoch 28, loss 3894262.25
epoch 29, loss 3893954.75
epoch 30, loss 3893648.5
epoch 31, loss 3893344.5
epoch 32, loss 3893041.25
epoch 33, loss 3892738.25
epoch 34, loss 3892436.0
epoch 35, loss 3892134.25
epoch 36, loss 3891832.0
epoch 37, loss 3891529.25
epoch 38, loss 3891226.0
epoch 39, loss 3890922.0
epoch 40, loss 389

epoch 457, loss 747478.75
epoch 458, loss 747276.3125
epoch 459, loss 747077.4375
epoch 460, loss 746882.4375
epoch 461, loss 746691.0
epoch 462, loss 746502.9375
epoch 463, loss 746318.25
epoch 464, loss 746136.75
epoch 465, loss 745958.9375
epoch 466, loss 745784.25
epoch 467, loss 745612.75
epoch 468, loss 745444.1875
epoch 469, loss 745278.5625
epoch 470, loss 745116.0625
epoch 471, loss 744956.5
epoch 472, loss 744799.625
epoch 473, loss 744645.3125
epoch 474, loss 744493.875
epoch 475, loss 744345.0625
epoch 476, loss 744198.6875
epoch 477, loss 744055.125
epoch 478, loss 743913.5625
epoch 479, loss 743774.75
epoch 480, loss 743638.375
epoch 481, loss 743504.25
epoch 482, loss 743372.5
epoch 483, loss 743242.9375
epoch 484, loss 743115.4375
epoch 485, loss 742990.1875
epoch 486, loss 742866.875
epoch 487, loss 742745.75
epoch 488, loss 742626.6875
epoch 489, loss 742509.6875
epoch 490, loss 742394.75
epoch 491, loss 742281.9375
epoch 492, loss 742170.75
epoch 493, loss 742061.625

epoch 924, loss 731057.25
epoch 925, loss 731042.5
epoch 926, loss 731027.4375
epoch 927, loss 731012.6875
epoch 928, loss 730997.75
epoch 929, loss 730983.0
epoch 930, loss 730968.0625
epoch 931, loss 730953.125
epoch 932, loss 730938.1875
epoch 933, loss 730923.375
epoch 934, loss 730908.4375
epoch 935, loss 730893.5625
epoch 936, loss 730878.5
epoch 937, loss 730863.625
epoch 938, loss 730848.625
epoch 939, loss 730833.625
epoch 940, loss 730818.75
epoch 941, loss 730803.625
epoch 942, loss 730788.6875
epoch 943, loss 730773.75
epoch 944, loss 730758.6875
epoch 945, loss 730743.625
epoch 946, loss 730728.625
epoch 947, loss 730713.3125
epoch 948, loss 730698.375
epoch 949, loss 730683.3125
epoch 950, loss 730668.25
epoch 951, loss 730653.0625
epoch 952, loss 730638.0625
epoch 953, loss 730622.8125
epoch 954, loss 730607.875
epoch 955, loss 730593.0
epoch 956, loss 730577.8125
epoch 957, loss 730562.6875
epoch 958, loss 730547.6875
epoch 959, loss 730532.5625
epoch 960, loss 730517.5

epoch 22280, loss 481757.0
epoch 22291, loss 481741.0625
epoch 22479, loss 481470.03125
epoch 22512, loss 481422.53125
epoch 22611, loss 481279.875
epoch 22837, loss 480953.84375
epoch 22876, loss 480897.21875
epoch 22967, loss 480764.6875
epoch 23102, loss 480567.15625
epoch 23219, loss 480395.25
epoch 23461, loss 480036.34375
epoch 23505, loss 479970.90625
epoch 23620, loss 479798.46875
epoch 23622, loss 479795.65625
epoch 23827, loss 479485.53125
epoch 23999, loss 479223.0
epoch 24002, loss 479218.40625
epoch 24142, loss 479003.3125
epoch 24272, loss 478802.09375
epoch 25318, loss 477143.09375
epoch 25407, loss 476999.90625
epoch 25425, loss 476971.0
epoch 25437, loss 476951.75
epoch 25498, loss 476853.3125
epoch 25511, loss 476832.34375
epoch 25561, loss 476751.46875
epoch 25573, loss 476732.0625
epoch 25605, loss 476680.21875
epoch 25999, loss 476037.25
epoch 26049, loss 475954.9375
epoch 26101, loss 475868.6875
epoch 26177, loss 475742.28125
epoch 26559, loss 475098.0
epoch 26603

...

epoch 182669, loss 85800.3828125
epoch 182672, loss 85798.7109375
epoch 182822, loss 85748.2890625
epoch 182838, loss 85743.15625
epoch 182846, loss 85740.875
epoch 183086, loss 85704.140625
epoch 183208, loss 85697.1640625
epoch 183218, loss 85699.3671875
epoch 183382, loss 85711.9921875
epoch 183417, loss 85704.9140625
epoch 183500, loss 85715.5703125
epoch 183789, loss 85724.34375
epoch 183791, loss 85724.859375
epoch 183991, loss 85718.7890625
epoch 184017, loss 85717.578125
epoch 184219, loss 85708.28125
epoch 184234, loss 85706.34375
epoch 184237, loss 85706.640625
epoch 184285, loss 85703.7421875
epoch 184290, loss 85703.109375
epoch 184516, loss 85681.046875
epoch 184531, loss 85680.4296875
epoch 184566, loss 85676.890625
epoch 184577, loss 85676.7890625
epoch 184622, loss 85673.046875
epoch 184710, loss 85668.34375
epoch 184798, loss 85664.5625
epoch 184826, loss 85663.4296875
epoch 184933, loss 85661.4296875
epoch 184956, loss 85660.25
epoch 185013, loss 85659.8515625
epoch 1

epoch 208872, loss 84698.359375
epoch 208879, loss 84698.1796875
epoch 208954, loss 84696.203125
epoch 209092, loss 84692.59375
epoch 209093, loss 84692.5546875
epoch 209125, loss 84691.734375
epoch 209225, loss 84690.40625
epoch 209264, loss 84689.9296875
epoch 209753, loss 84678.3359375
epoch 209774, loss 84677.859375
epoch 209840, loss 84676.125
epoch 209846, loss 84675.9765625
epoch 210358, loss 84661.703125
epoch 210574, loss 84657.0390625
epoch 210606, loss 84656.203125
epoch 210714, loss 84653.671875
epoch 210778, loss 84652.0390625
epoch 210836, loss 84650.640625
epoch 210871, loss 84649.703125
epoch 211033, loss 84645.890625
epoch 211145, loss 84643.1640625
epoch 211421, loss 84636.0390625
epoch 211472, loss 84634.875
epoch 211620, loss 84631.046875
epoch 211658, loss 84628.7265625
epoch 211730, loss 84625.0546875
epoch 211777, loss 84623.46875
epoch 211923, loss 84619.4453125
epoch 211961, loss 84618.4609375
epoch 212060, loss 84615.9140625
epoch 212164, loss 84613.234375
epo

epoch 234158, loss 84087.5625
epoch 234161, loss 84087.5546875
epoch 234170, loss 84087.3125
epoch 234198, loss 84086.703125
epoch 234234, loss 84085.765625
epoch 234296, loss 84084.3984375
epoch 234469, loss 84080.5078125
epoch 234632, loss 84076.8359375
epoch 234692, loss 84075.5625
epoch 234732, loss 84074.7421875
epoch 234771, loss 84073.8046875
epoch 234825, loss 84072.59375
epoch 234840, loss 84072.1875
epoch 234928, loss 84070.34375
epoch 234960, loss 84069.5234375
epoch 235039, loss 84067.7578125
epoch 235172, loss 84064.84375
epoch 235323, loss 84061.453125
epoch 235601, loss 84055.234375
epoch 235671, loss 84053.78125
epoch 235678, loss 84053.5078125
epoch 235874, loss 84049.1015625
epoch 235988, loss 84046.5859375
epoch 236024, loss 84045.75
epoch 236191, loss 84042.1875
epoch 236257, loss 84040.6328125
epoch 236393, loss 84037.65625
epoch 236648, loss 84031.984375
epoch 236682, loss 84031.1015625
epoch 236836, loss 84027.6328125
epoch 236847, loss 84027.4453125
epoch 236891

epoch 259420, loss 83572.0625
epoch 259466, loss 83571.2265625
epoch 259476, loss 83571.0234375
epoch 259839, loss 83564.28125
epoch 259863, loss 83563.8359375
epoch 260168, loss 83558.140625
epoch 260348, loss 83554.8125
epoch 260618, loss 83549.8046875
epoch 260653, loss 83549.15625
epoch 260673, loss 83548.7890625
epoch 260801, loss 83546.3984375
epoch 260832, loss 83545.8359375
epoch 260921, loss 83544.1875
epoch 260922, loss 83544.1640625
epoch 261022, loss 83542.328125
epoch 261083, loss 83541.203125
epoch 261117, loss 83540.5703125
epoch 261475, loss 83533.9296875
epoch 261505, loss 83533.375
epoch 261754, loss 83528.734375
epoch 261873, loss 83526.5390625
epoch 262069, loss 83522.9375
epoch 262169, loss 83521.0859375
epoch 262235, loss 83519.8828125
epoch 262265, loss 83519.328125
epoch 262332, loss 83518.1015625
epoch 262409, loss 83516.6953125
epoch 262430, loss 83516.296875
epoch 262460, loss 83515.7578125
epoch 262513, loss 83514.78125
epoch 262594, loss 83513.296875
epoch 

epoch 286295, loss 83115.6328125
epoch 286808, loss 83107.8125
epoch 287301, loss 83100.3125
epoch 287332, loss 83099.84375
epoch 287498, loss 83097.328125
epoch 287509, loss 83097.15625
epoch 287644, loss 83095.109375
epoch 287716, loss 83094.015625
epoch 287791, loss 83092.8671875
epoch 288004, loss 83089.6484375
epoch 288096, loss 83088.2578125
epoch 288192, loss 83086.8125
epoch 288347, loss 83084.453125
epoch 288716, loss 83078.875
epoch 288813, loss 83077.40625
epoch 288841, loss 83076.984375
epoch 288909, loss 83075.984375
epoch 288940, loss 83075.5078125
epoch 288973, loss 83075.015625
epoch 289066, loss 83073.609375
epoch 289106, loss 83073.015625
epoch 289151, loss 83072.34375
epoch 289285, loss 83070.3359375
epoch 289316, loss 83069.859375
epoch 289502, loss 83067.078125
epoch 289524, loss 83066.734375
epoch 289630, loss 83065.1640625
epoch 289685, loss 83064.3359375
epoch 289954, loss 83060.328125
epoch 290021, loss 83059.3203125
epoch 290248, loss 83055.90625
epoch 290276,